In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler   # u otros scalers
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from distancia import haversine

In [ ]:
### carga datos de dataset en dataframe
file_path= 'uber_fares.csv'

df = pd.read_csv(file_path)

In [ ]:
### visualizacion de algunos datos
df.head()

#### Contexto  
El proyecto trata sobre **Uber Inc.**, la compañía de taxis más grande del mundo. En este trabajo, nuestro objetivo es **predecir la tarifa de futuros viajes**.  

Uber brinda servicio a millones de clientes cada día, por lo que gestionar adecuadamente sus datos es clave para desarrollar nuevas estrategias de negocio y obtener mejores resultados.  

### Variables del conjunto de datos  

**Variables explicativas:**  
- **key**: identificador único de cada viaje.  
- **pickup_datetime**: fecha y hora en que se inició el viaje.  
- **passenger_count**: cantidad de pasajeros en el vehículo (dato ingresado por el conductor).  
- **pickup_longitude**: longitud del punto de inicio del viaje.  
- **pickup_latitude**: latitud del punto de inicio del viaje.  
- **dropoff_longitude**: longitud del punto de destino.  
- **dropoff_latitude**: latitud del punto de destino.  

**Variable objetivo (target):**  
- **fare_amount**: costo del viaje en dólares.  

In [ ]:
### Columnas, ¿cuáles son variables numéricas y cuales variables categóricas?
df.columns

En este dataset trabajaremos con algunas columnas de interes, las cuales clasificamos a continuacion dependiendo el tipo de variable:

*   **fare_amount:**   cuantitativa continua
*   **pickup_datetime:** cualitativa nominal
*   **pickup_longitude:** cuantitativa continua
*   **pickup_latitude:** cuantitativa continua
*   **dropoff_longitude:** cuantitativa continua
*   **dropoff_latitude:** cuantitativa continua
*   **passenger_count:** cuantitativa discreta

Lo primero que realizaremos será utilizar el método `.info()` para verificar que el tipo de dato en cada variable es correcto, detectar la presencia de valores nulos y valores atípicos.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns='fare_amount'), df['fare_amount'], test_size=0.2, random_state=42)

In [ ]:
X_train.info()

In [ ]:
# mostrar la fila con valores nulos
X_train[X_train.isnull().any(axis=1)]

### Limpieza de los datos
Podemos observar a traves del 'X_train.info' que tenemos 200.000 datos donde las columnas 'dropoff_longitude' y 'dropoff_latitude' poseen un valor faltante en sus datos. Consideramos que al tener una muestra de datos abundante, decidimos eliminarlos del dataset ya que no afecta al entrenamiento del modelo y evitar imputar los datos con un valor promedio. Además, modificamos el tipo de dato de 'pickup_datetime' para verificar que todas las fechas ingresadas hayan sido correctamente cargadas.

In [ ]:
# Cambia el tipo de dato object -> datetime
X_train['pickup_datetime'] = pd.to_datetime(X_train['pickup_datetime'])                   
# Elimina las filas con valores nulos
#X_train.dropna(inplace=True)     #No haria falta lo hacemos con imputacion

# Eliminación de columnas que no son de interes
X_train = X_train.drop('key', axis = 1)
X_train = X_train.drop('date', axis = 1)

Luego de esto, realizamos un '.describe()' sobre las variables numericas y consideramos los resultados para analizar cada variable del dataset.

In [ ]:
X_train.describe()

##### **Análisis de la columna `passenger_count`:**

A partir del análisis exploratorio mediante `.describe()`, se detectó que el valor máximo registrado en la variable fue de 208 pasajeros por viaje, lo cual constituye un error en la carga de datos. Para visualizar la distribución se realizaron diferentes gráficos:  

- En el **primer boxplot**, se observa claramente el valor atípico de 208, lo que impide una correcta interpretación del resto de los datos.  
- Al excluir dicho valor extremo, el **segundo boxplot** permite observar mejor la dispersión real de la variable.  

Además, se identificaron registros con 0 pasajeros, los cuales también son inconsistentes, ya que un viaje no puede realizarse sin pasajeros.  


<span style="color: red;">quizas deberiamos calcular los valores atipicos con q1 y q3 como en distancia??</span>

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
sns.boxplot(x = X_train['passenger_count'])
plt.title('Boxplot de cantidad de pasajeros sin filtrar')
plt.xlabel('Cantidad de pasajeros')

plt.subplot(1, 2, 2)
sns.boxplot(x = X_train[X_train['passenger_count'] < 208]['passenger_count'])
plt.title('Boxplot de cantidad de pasajero filtrado')
plt.xlabel('Cantidad de pasajeros')

plt.show()

In [ ]:
plt.hist(X_train[X_train['passenger_count'] < 208]['passenger_count'], color = 'green')
plt.title('Histograma de cantidad de pasajeros menor a 208')
plt.xlabel('Cantidad de pasajeros')
plt.ylabel('Frecuencia')
plt.show()

Para solucionar este problema y no eliminar dichos registros, se decidió imputar el valor atípico de 208 y tratar los datos con 0 pasajeros con la **mediana de la distribución**. La mediana es la medida de tendencia central y es robusta frente a valores atípicos y representa de forma adecuada la cantidad más común de pasajeros en los viajes (generalmente 1). Finalmente, la columna fue convertida a valores enteros para mantener consistencia en la variable.


In [ ]:
from sklearn.impute import SimpleImputer

X_train.loc[X_train['passenger_count'] == 208, 'passenger_count'] = pd.NA
X_train.loc[X_train['passenger_count'] == 0, 'passenger_count'] = pd.NA
imputer = SimpleImputer(strategy='median')                                      # Imputar con la mediana
X_train['passenger_count'] = imputer.fit_transform(X_train[['passenger_count']])
X_train['passenger_count'] = X_train['passenger_count'].astype(int)

In [ ]:
plt.hist(X_train['passenger_count'], color = 'green')
plt.title('Histograma de cantidad de pasajeros transformado')
plt.xlabel('Cantidad de pasajeros')
plt.ylabel('Frecuencia')
plt.show()

##### Análisis de las columnas `pickup_latitude`, `dropoff_latitude`, `pickup_longitude` y `dropoff_longitude`

Para analizar estos datos, lo primero que realizamos fue la visualización gráfica de cada variable. En estas gráficas se observan bastantes valores atípicos que no corresponden a rangos válidos de latitud y longitud:  

- **Rangos globales válidos**:  
  - Latitud: -90 a 90  
  - Longitud: -180 a 180  

Vamos a considerar las coordenadas que caigan dentro de estos rangos o bien tengan una sola coordenada de las 4 posibles fuera del rango.
En estos casos imputariamos la coordenada invalida con KNN

<span style="color: red;">Esto no lo pondria, porque esto seria una suposicion nuestra</span>
- **Rangos aproximados para New York**:  
  - Latitud: 40.50 a 41.00  
  - Longitud: -74.20 a -73.60  

De esta manera, se eliminan registros con coordenadas fuera de este rango, ya que son considerados como datos no representativos de la zona de análisis.

In [ ]:
plt.figure(figsize=(24,5))

plt.subplot(1, 4, 1)
plt.title("Datos de 'pickup_longitude'")
sns.boxplot(X_train['pickup_longitude'])
plt.ylabel('Longitud de inicio')

plt.subplot(1, 4, 2)
plt.title("Datos de 'dropoff_longitude'")
sns.boxplot(X_train['dropoff_longitude'])
plt.ylabel('Longitud final')

plt.subplot(1, 4, 3)
plt.title("Datos de 'pickup_latitude'")
sns.boxplot(X_train['pickup_latitude'])
plt.ylabel('Latitud de inicio')

plt.subplot(1, 4, 4)
plt.title("Datos de 'dropoff_latitude'")
sns.boxplot(X_train['dropoff_latitude'])
plt.ylabel('Latitud final')

plt.show()

In [ ]:
# Esto creo que lo podemos quitar

# longitud = df[(df['pickup_longitude'] == 0.00) & (df['dropoff_longitude'] == 0.00)].value_counts().sum()  # 3587 datos
# latitud = df[(df['pickup_longitude'] == 0.00) & (df['dropoff_longitude'] == 0.00)].value_counts().sum()   # 3587 datos
# lat_long = df[(df['pickup_latitude'] == 0.00) & (df['dropoff_latitude'] == 0.00) & (df['pickup_longitude'] == 0.00) & (df['dropoff_longitude'] == 0.00)].value_counts().sum()
# print(f'datos nulos en latitud y longitud: {lat_long}') # 3587 datos
# print(f'datos nulos en longitud: {longitud}')
# print(f'datos nulos en latitud: {latitud}\n')

# longitud1 = df[df['dropoff_longitude'] == 0.00].value_counts().sum()  # 177 datos
# longitud2 = df[df['pickup_longitude'] == 0.00].value_counts().sum()   # 199 datos 
# latitud1 = df[(df['pickup_latitude'] == 0.00)].value_counts().sum()   # 177 datos 
# latitud2 = df[(df['dropoff_latitude'] == 0.00)].value_counts().sum()  # 171 datos 
# print(f'datos nulos en longitud inicio: {longitud2}')
# print(f'datos nulos en longitud final: {longitud1}')
# print(f'datos nulos en latitud inicio: {latitud1}')
# print(f'datos nulos en latitud final: {latitud2}\n')

Podemos observar que existen registros en los cuales tanto la latitud como la longitud (de inicio y de destino) tienen valor igual a 0. Estos registros no aportan información relevante al análisis, ya que corresponden a errores de carga, por lo que se decidió **eliminarlos del dataset** para trabajar únicamente con datos reales.  

Por otro lado, aquellos registros en los que solo **una de las coordenadas** (latitud o longitud, ya sea en inicio o destino) tiene valor igual a 0, pueden ser imputados por aproximación. En este caso, en lugar de eliminarlos, se optó por reemplazar dichos valores utilizando el método de imputación por K-vecinos más cercanos (KNN), lo cual permite estimar las coordenadas faltantes en base a los viajes más similares.

In [ ]:
# X_train[(X_train['pickup_latitude'] == 0.00) &(X_train['dropoff_latitude'] == 0.00) &(X_train['pickup_longitude'] == 0.00) &(X_train['dropoff_longitude'] == 0.00)].head()

In [ ]:
# Eliminar filas con todas las coordenadas en 0
#indices_invalidos = X_train[(X_train['pickup_latitude'] == 0.00) &(X_train['dropoff_latitude'] == 0.00) &(X_train['pickup_longitude'] == 0.00) &(X_train['dropoff_longitude'] == 0.00)].index
#X_train = X_train.drop(indices_invalidos)
# Cantidad de filas eliminadas
#print(f'Cantidad de filas eliminadas: {len(indices_invalidos)}')

In [ ]:
# recorrer el dataset X_train y chequear si las coordenadas son validas o cero o nan y si no los son reemplazarlas con NaN
# si una fila tiene mas de 1 coordenada invalida, eliminar la fila
# mostrar la cantidad de filas eliminadas
# mostrar la cantidad de filas modificadas a NaN

modified_count = 0
initial_rows = X_train.shape[0] # cantidad inicial de filas
for index, row in X_train.iterrows():
    invalid_count = 0

    if not (-90 <= row['pickup_latitude'] <= 90) or row['pickup_latitude'] == 0.0 or pd.isna(row['pickup_latitude']): 
        X_train.at[index, 'pickup_latitude'] = pd.NA
        invalid_count += 1
    if not (-90 <= row['dropoff_latitude'] <= 90 or row['dropoff_latitude'] == 0.0) or pd.isna(row['dropoff_latitude']):
        X_train.at[index, 'dropoff_latitude'] = pd.NA
        invalid_count += 1
    if not (-180 <= row['pickup_longitude'] <= 180) or row['pickup_longitude'] == 0.0 or pd.isna(row['pickup_longitude']):
        X_train.at[index, 'pickup_longitude'] = pd.NA
        invalid_count += 1
    if not (-180 <= row['dropoff_longitude'] <= 180) or row['dropoff_longitude'] == 0.0 or pd.isna(row['dropoff_longitude']):
        X_train.at[index, 'dropoff_longitude'] = pd.NA
        invalid_count += 1
    if invalid_count > 1:
        X_train = X_train.drop(index)
        y_train = y_train.drop(index)
    elif invalid_count == 1:
        modified_count += 1

print(f'Cantidad de filas modificadas a NaN: {modified_count}')
final_rows = X_train.shape[0] # cantidad final de filas
print(f'Cantidad de filas eliminadas: {initial_rows - final_rows}')

In [ ]:
from sklearn.impute import KNNImputer

columnas = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
#k muy pequeño → más sensible a ruido. k muy grande → valores muy suavizados, se pierde variabilidad real.

# Crear imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)
X_train[columnas] = knn_imputer.fit_transform(X_train[columnas])

In [ ]:
plt.figure(figsize=(24,5))

plt.subplot(1, 4, 1)
plt.title("Datos de 'pickup_longitude'")
sns.boxplot(X_train['pickup_longitude'], color="skyblue")
plt.ylabel('Longitud de inicio')


plt.subplot(1, 4, 2)
plt.title("Datos de 'dropoff_longitude'")
sns.boxplot(X_train['dropoff_longitude'], color="pink")
plt.ylabel('Longitud final')

plt.subplot(1, 4, 3)
plt.title("Datos de 'pickup_latitude'")
sns.boxplot(X_train['pickup_latitude'], color="skyblue")
plt.ylabel('Latitud de inicio')

plt.subplot(1, 4, 4)
plt.title("Datos de 'dropoff_latitude'")
sns.boxplot(X_train['dropoff_latitude'], color="pink")
plt.ylabel('Latitud final')

plt.show()

In [ ]:
# Agregar mas caraterísticas a partir de las coordenadas geográficas
X_train["delta_lat"] = X_train["dropoff_latitude"] - X_train["pickup_latitude"]
X_train["delta_lon"] = X_train["dropoff_longitude"] - X_train["pickup_longitude"]

X_train["distance_km"] = X_train.apply(
    lambda row: haversine(
        row["pickup_longitude"], row["pickup_latitude"],
        row["dropoff_longitude"], row["dropoff_latitude"]
    ),
    axis=1
)
X_train.head()

In [ ]:
X_train.describe()

In [ ]:
# histograma de la distancia en km

plt.hist(X_train['distance_km'], bins=50, color='blue', alpha=0.7)
plt.title('Histograma de la distancia en km')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:
# histograma del logaritmo de la distancia en km
plt.hist(np.log1p(X_train['distance_km']), bins=50, color='blue', alpha=0.7)
plt.title('Histograma del logaritmo de la distancia en km')
plt.xlabel('Log(Distancia + 1)')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:
# calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
Q1 = X_train['distance_km'].quantile(0.25)
Q3 = X_train['distance_km'].quantile(0.75)
IQR = Q3 - Q1
# definir los límites para los valores atípicos
lower_bound = Q1 - 5 * IQR
upper_bound = Q3 + 5 * IQR
print(f'Q1: {Q1}, Q3: {Q3}, IQR: {IQR}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}')

In [ ]:
# calcular la cantidad de valores atípicos
outliers = X_train[(X_train['distance_km'] < lower_bound) | (X_train['distance_km'] > upper_bound)]
print(f'Cantidad de valores atípicos: {outliers.shape[0]}')
print(f'Porcentaje de valores atípicos: {outliers.shape[0] / X_train.shape[0] * 100:.2f}%')

In [ ]:
# obtener index de los datos para eliminar los valores atípicos
outlier_indices = outliers.index
# eliminar los datos atípicos de x_train y y_train
y_train = y_train.drop(outlier_indices)
X_train = X_train.drop(outlier_indices)


plt.hist(X_train['distance_km'], bins=50, color='blue', alpha=0.7)
plt.title('Histograma de la distancia en km')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:
y_train.describe()

In [ ]:
# histograma de y_train
plt.hist(y_train, bins=50, color='orange', alpha=0.7)
plt.title('Histograma de y_train (fare_amount)')
plt.xlabel('Fare Amount')
plt.ylabel('Frecuencia')        

In [ ]:
X_train.info()

In [ ]:
y_train.info()

In [ ]:
# mostrar cantidad de viajes por año y mes
X_train['pickup_year'] = X_train['pickup_datetime'].dt.year
X_train['pickup_month'] = X_train['pickup_datetime'].dt.month
# mostrar un grafico de barras con la cantidad de viajes por año y mes
plt.figure(figsize=(12,6))
sns.countplot(data=X_train, x='pickup_year', hue='pickup_month', palette='viridis')
plt.title('Cantidad de viajes por año y mes')
plt.xlabel('Año')
plt.ylabel('Cantidad de viajes')
plt.legend(title='Mes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()  


In [ ]:
# obtener el dia de la semana y la hora del dia a partir de pickup_datetime
X_train['pickup_day_of_week'] = X_train['pickup_datetime'].dt.dayofweek
X_train['pickup_hour'] = X_train['pickup_datetime'].dt.hour
X_train.head()

In [ ]:
# agregar campo franja horaria a partir del campo pickup_hour
def franja_horaria(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Mañana'
    elif 12 <= hora < 13:
        return 'Mediodia'
    elif 13 <= hora < 18:
        return 'Tarde'
    elif 18 <= hora < 24:
        return 'Noche'
    else:
        return 'Desconocido'
X_train['franja_horaria'] = X_train['pickup_hour'].apply(franja_horaria)
X_train.head()

In [ ]:
# mostrar un scatter plot de la distancia en km vs fare_amount, color franja horaria
plt.figure(figsize=(10,6))
sns.scatterplot(data=X_train, x='distance_km', y=y_train, hue='franja_horaria', palette='viridis', alpha=0.6)
plt.title('Distancia en km vs Fare Amount por Franja Horaria')
plt.xlabel('Distancia (km)')
plt.ylabel('Fare Amount')
plt.legend(title='Franja Horaria', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()



In [ ]:
# mostrar un scatter plot de la distancia en km vs fare_amount, color dia de la semana
plt.figure(figsize=(10,6))
sns.scatterplot(data=X_train, x='distance_km', y=y_train, hue='pickup_day_of_week', palette='viridis', alpha=0.6)
plt.title('Distancia en km vs Fare Amount por pickup_day_of_week')
plt.xlabel('Distancia (km)')
plt.ylabel('Fare Amount')
plt.legend(title='pickup_day_of_week', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()